In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.jars", "/home/liangcail/work/files/jars/cudf.jar,/home/liangcail/work/files/jars/rapids.jar,/home/liangcail/work/files/jars/udf.jar") \
    .config("spark.sql.adaptive.enabled", "false") \
    .config("spark.default.parallelism", "3") \
    .config("spark.executor.memory", "50GB") \
    .config("spark.executor.cores", "6") \
    .config("spark.plugins", "com.nvidia.spark.SQLPlugin") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("park.rapids.sql.batchSizeBytes", "1GB") \
    .config("spark.executor.resource.gpu.amount", "1") \
    .getOrCreate()
spark
# register the udf
spark.udf.registerJavaFunction("point_in_polygon", "com.nvidia.spark.rapids.udf.PointInPolygon", None)

In [ ]:
# prepare shape files
spark.sparkContext.addFile("/data/cuspatial/taxi_zones/shape/taxi_zones.shp")
spark.sparkContext.addFile("/data/cuspatial/taxi_zones/shape/taxi_zones.shx")

spark.sparkContext.addFile("/data/cuspatial/taxi_zones/shape/nyct2000.shp")
spark.sparkContext.addFile("/data/cuspatial/taxi_zones/shape/nyct2000.shx")

ptPath = "/data/cuspatial/taxi_zones/pt/"
outPath = "/data/cuspatial/out/gpu/"

In [ ]:
# set its parameters via SQL config for runtime updating
spark.conf.set("spark.cuspatial.sql.udf.shapeFileName", "nyct2000.shp")

In [ ]:
# read the points data
df = spark.read.parquet(ptPath)
df = df.selectExpr('x', 'y', 'point_in_polygon(x, y) as point_in_polygon')

In [ ]:
import time

In [ ]:
begin = time.time()
df.write.mode("overwrite").parquet(outPath)
end = time.time()
print("==> It took {} s".format(round(end-begin, 2)))